In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot

In [26]:
# load the dataset as a dataframe using pd
data = pd.read_csv('C:/Users/Anusha/Desktop/ExcelRProjects/dataset/Fraud_check.csv')
data.head(5)

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


## Initial EDA

In [27]:
##As mentioned in the business Converting the Taxable income variable to  Income variable by bucketing.
data.loc[data["Taxable.Income"]>=30000,"income"]="Good"
data.loc[data["Taxable.Income"]<=30000,"income"]="Risky"

In [28]:
data.head(5)

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,income
0,NO,Single,68833,50047,10,YES,Good
1,YES,Divorced,33700,134075,18,YES,Good
2,NO,Married,36925,160205,30,YES,Good
3,YES,Single,50190,193264,15,YES,Good
4,NO,Married,81002,27533,28,NO,Good


#### Finding Correlation b/w different variables for the given dataset

In [30]:
import ppscore as pps
pps.matrix(data)

,x,y,ppscore,case,is_valid_score,metric,baseline_score,model_score,model
0,Undergrad,Undergrad,1.000000,predict_itself,True,None,0.000000,1.000000,None
1,Undergrad,Marital.Status,0.000000,classification,True,weighted F1,0.333333,0.211271,DecisionTreeClassifier()
2,Undergrad,Taxable.Income,0.000000,regression,True,mean absolute error,22771.961667,22779.973615,DecisionTreeRegressor()
3,Undergrad,City.Population,0.006556,regression,True,mean absolute error,42730.781667,42450.626013,DecisionTreeRegressor()
4,Undergrad,Work.Experience,0.000000,regression,True,mean absolute error,7.598333,7.710043,DecisionTreeRegressor()
5,Undergrad,Urban,0.000000,classification,True,weighted F1,0.493333,0.437896,DecisionTreeClassifier()
6,Undergrad,income,0.000000,classification,True,weighted F1,0.701908,0.701908,DecisionTreeClassifier()
7,Marital.Status,Undergrad,0.000000,classification,True,weighted F1,0.513333,0.464649,DecisionTreeClassifier()
8,Marital.Status,Marital.Status,1.000000,predict_itself,True,None,0.000000,1.000000,None
9,Marital.Status,Taxable.Income,0.007396,regression,True,mean absolute error,22771.961667,22603.545807,DecisionTreeRegressor()


###### From the above pps score matrix we can conclude there is no collinearity problem existing b/w in put variables
###### and taxable.income is highly required for predicting income variable, maritial status also helps in predicting income variable etc

In [29]:
data['income'].value_counts()

Good     476
Risky    124
Name: income, dtype: int64

In [16]:
##As, the model cannot be applied directly on object data we need to encode this object variables
def encode_labels(dataset):
    from sklearn import preprocessing 
    le = preprocessing.LabelEncoder()
    for column_name in dataset.columns:
        if dataset[column_name].dtype == object:
            dataset[column_name] = le.fit_transform(dataset[column_name])
        else:
            pass

In [9]:
encode_labels(data)

In [10]:
data.head(5)

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,income
0,0,2,68833,50047,10,1,0
1,1,0,33700,134075,18,1,0
2,0,1,36925,160205,30,1,0
3,1,2,50190,193264,15,1,0
4,0,1,81002,27533,28,0,0


###### income 0 indicates good, 1 indicates risky
###### undergrad 0 indicates graduation not completed,1 indicates graduated/educated
###### 2- single,0 - divorced,1-married

## Model Building using Random Forest Classifier

###### As missing values,best feature selection, overfitting problems can be handled by random forest classifier
###### we can start building the model using Random Forest

In [13]:
# Importing packages required for model building and validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [14]:
# Grid Search for Algorithm Tuning
from sklearn.model_selection import GridSearchCV
import numpy

In [32]:
max_features = numpy.array(range(1,6))
num_trees = numpy.array(range(50,1000,50))
param_grid = dict(n_estimators=num_trees,max_features=max_features)

In [33]:
param_grid

{'n_estimators': array([ 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650,
        700, 750, 800, 850, 900, 950]),
 'max_features': array([1, 2, 3, 4, 5])}

In [39]:
array=data.values 
X = array[:,0:6]
Y = array[:,6]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

In [40]:
model = RandomForestClassifier()
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid.fit(X, Y)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_features': array([1, 2, 3, 4, 5]),
                         'n_estimators': array([ 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650,
       700, 750, 800, 850, 900, 950])})

In [41]:
print(grid.best_score_)

1.0


In [42]:
print(grid.best_params_)

{'max_features': 1, 'n_estimators': 300}


In [43]:
## building a random forest with best no. max features as 1 and no. of trees as 150
## evaluating the model using Kfold crossvalidation technique
num_trees = 150
max_features = 1
kfold = KFold(n_splits=10, random_state=7)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

C:\Users\Anusha\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


0.9983333333333334


### Conclusion:
##### As we know Income values to totaly depend on Taxable.Income feature, As we have created the y label Income by bucketing Taxable.Income variable, so  Taxable income is completely sufficent for pretecting the Income value, even from the above model we got the same conclusion, 1- best feature is required for prediction of ylable and accuracy is nearly 100% as y is completely in linear relation with taxable.income feature

### Implementing the model by removing Taxable.Income feature

In [11]:
# load the dataset as a dataframe using pd
data = pd.read_csv('C:/Users/Anusha/Desktop/ExcelRProjects/dataset/Fraud_check.csv')

In [12]:
##As mentioned in the business Converting the Taxable income variable to  Income variable by bucketing.
data.loc[data["Taxable.Income"]>=30000,"income"]="Good"
data.loc[data["Taxable.Income"]<=30000,"income"]="Risky"

In [13]:
data['income'].value_counts()

Good     476
Risky    124
Name: income, dtype: int64

In [14]:
##Droping the Taxable income variable
data.drop(["Taxable.Income"],axis=1,inplace=True)

In [17]:
encode_labels(data)

## Model Building using Random Forest

In [18]:
# Importing packages required for model building and validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
# Grid Search for Algorithm Tuning
from sklearn.model_selection import GridSearchCV
import numpy

In [19]:
max_features = numpy.array(range(1,5))
num_trees = numpy.array(range(50,1000,50))
param_grid = dict(n_estimators=num_trees,max_features=max_features)

In [20]:
param_grid

{'n_estimators': array([ 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650,
        700, 750, 800, 850, 900, 950]),
 'max_features': array([1, 2, 3, 4])}

In [21]:
array=data.values 
X = array[:,0:5]
Y = array[:,5]

In [56]:
model = RandomForestClassifier()
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid.fit(X, Y)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_features': array([1, 2, 3, 4]),
                         'n_estimators': array([ 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650,
       700, 750, 800, 850, 900, 950])})

In [57]:
print(grid.best_score_)

0.7516666666666667


In [58]:
print(grid.best_params_)

{'max_features': 3, 'n_estimators': 350}


In [22]:
## building a random forest with best no. max features as 3 and no. of trees as 350
## evaluating the model using Kfold crossvalidation technique
num_trees = 350
max_features = 3
kfold = KFold(n_splits=10, random_state=7)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

C:\Users\Anusha\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


0.7416666666666666


## Conclusion
###### The maximum accuracy that we can get for the above model is 74.16 % as we have droped the Taxable income fature which is highly need for predicting the Income value, with other features which are not highly correlated with output value can only give the max accuracy of 74%, from this we can conclude selecting the best feature can change the models accuracy drastically